In [1]:
import cv2
import os
import re

# Konfigurasi
image_folder = 'D:\\TA_Lalu_Lintas\\TA-Lalu-Lintas\\dataset-mentah\\images2'  # Path ke folder gambar
output_yolo_images = 'dataset_yolo/images/'
output_yolo_labels = 'dataset_yolo/labels/'
output_cnn_worn = 'dataset_cnn/helmet_worn/'
output_cnn_not_worn = 'dataset_cnn/helmet_not_worn/'
os.makedirs(output_yolo_images, exist_ok=True)
os.makedirs(output_yolo_labels, exist_ok=True)
os.makedirs(output_cnn_worn, exist_ok=True)
os.makedirs(output_cnn_not_worn, exist_ok=True)

# Variabel Global
drawing = False
ix, iy = -1, -1
boxes = []
current_class = 0
working_img = None  # Tambahkan variabel working image

# Fungsi menggambar bounding box
def draw_bbox(event, x, y, flags, param):
    global ix, iy, drawing, working_img, boxes, current_class

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        ix, iy = x, y

    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            temp_img = working_img.copy()
            color = (0, 255, 0) if current_class == 0 else (255, 0, 0)
            cv2.rectangle(temp_img, (ix, iy), (x, y), color, 2)
            cv2.imshow('Image', temp_img)

    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        color = (0, 255, 0) if current_class == 0 else (255, 0, 0)
        # Gambar langsung ke working image
        cv2.rectangle(working_img, (ix, iy), (x, y), color, 2)
        x_min, y_min = min(ix, x), min(iy, y)
        x_max, y_max = max(ix, x), max(iy, y)  # Perbaikan typo di sini
        boxes.append((current_class, x_min, y_min, x_max, y_max))
        cv2.imshow('Image', working_img)

# Fungsi untuk mendapatkan nomor urut terakhir dari file dalam sebuah folder
def get_last_file_number(folder_path, prefix, extension):
    files = os.listdir(folder_path)
    pattern = re.compile(f'{prefix}_(\\d+){extension}')
    numbers = [int(pattern.match(file).group(1)) for file in files if pattern.match(file)]
    return max(numbers) if numbers else 0

# Fungsi utama
image_files = [f for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]

for image_file in image_files:
    img_path = os.path.join(image_folder, image_file)
    img = cv2.imread(img_path)
    working_img = img.copy()  # Inisialisasi working image
    original_img = img.copy()  # Simpan gambar asli
    boxes = []

    cv2.namedWindow('Image')
    cv2.setMouseCallback('Image', draw_bbox)

    while True:
        cv2.imshow('Image', working_img)
        key = cv2.waitKey(1)

        if key == ord('m'):
            current_class = 0
        elif key == ord('h'):
            current_class = 1
        elif key == ord('r'):  # Reset
            working_img = original_img.copy()
            boxes.clear()
        elif key == 13:  # Enter
            if not boxes:
                print(f"Tidak ada label pada gambar {image_file}, tidak disimpan!")
                break
            
            # Simpan label dalam format YOLO
            last_label_number = get_last_file_number(output_yolo_labels, "frame", ".txt")
            label_file = os.path.join(output_yolo_labels, f'frame_{last_label_number + 1:04d}.txt')

            with open(label_file, 'w') as f:
                motor_bboxes = []
                helmet_bboxes = []
                for class_id, x_min, y_min, x_max, y_max in boxes:
                    x_center = (x_min + x_max) / 2 / img.shape[1]
                    y_center = (y_min + y_max) / 2 / img.shape[0]
                    width = (x_max - x_min) / img.shape[1]
                    height = (y_max - y_min) / img.shape[0]
                    f.write(f'{class_id} {x_center} {y_center} {width} {height}\n')

                    if class_id == 0:
                        motor_bboxes.append((x_min, y_min, x_max, y_max))
                    elif class_id == 1:
                        helmet_bboxes.append((x_min, y_min, x_max, y_max))

            # Simpan gambar ASLI tanpa bounding box ke folder YOLO
            last_image_number = get_last_file_number(output_yolo_images, "frame", ".jpg")
            image_file_output = os.path.join(output_yolo_images, f'frame_{last_image_number + 1:04d}.jpg')
            cv2.imwrite(image_file_output, original_img)

            # Ekstrak kepala untuk dataset CNN
            for motor_bbox in motor_bboxes:
                x_min, y_min, x_max, y_max = motor_bbox
                head_crop = img[y_min:y_min + int(0.3 * (y_max - y_min)), x_min:x_max]  # Ambil dari img asli

                helmet_found = False
                for helmet_bbox in helmet_bboxes:
                    hx_min, hy_min, hx_max, hy_max = helmet_bbox
                    if hx_min >= x_min and hy_min >= y_min and hx_max <= x_max and hy_max <= y_max:
                        helmet_found = True
                        break

                if helmet_found:
                    last_worn = get_last_file_number(output_cnn_worn, "head", ".jpg") + 1
                    cv2.imwrite(os.path.join(output_cnn_worn, f'head_{last_worn:04d}.jpg'), head_crop)
                else:
                    last_not_worn = get_last_file_number(output_cnn_not_worn, "head", ".jpg") + 1
                    cv2.imwrite(os.path.join(output_cnn_not_worn, f'head_{last_not_worn:04d}.jpg'), head_crop)

            print(f"Label dan gambar untuk {image_file} telah disimpan.")
            break

        elif key == ord('q'):  # Keluar program
            print("Proses dihentikan oleh pengguna.")
            cv2.destroyAllWindows()
            exit()

    cv2.destroyWindow('Image')


Label dan gambar untuk 404_jpg.rf.967b270686c557152c88701b553ad7e7.jpg telah disimpan.
Label dan gambar untuk 404_jpg.rf.c180973b002d9d4cc948b2950e33d84b.jpg telah disimpan.
Label dan gambar untuk 407_jpg.rf.9d01042dec580fee52a0b5ef36dc846b.jpg telah disimpan.
Label dan gambar untuk 409_jpg.rf.da038bfa37680e04232781f716692c92.jpg telah disimpan.
Label dan gambar untuk 427_jpg.rf.a04b4d4d088d61bc190756709af40401.jpg telah disimpan.
Label dan gambar untuk 427_jpg.rf.e6ee1474b3bc5ddd2b3606dd3c97d3ee.jpg telah disimpan.
Label dan gambar untuk 428_jpg.rf.4e0939f15b6494d0a4f5564ab648895f.jpg telah disimpan.
Label dan gambar untuk 451_jpg.rf.26d6a33959017bc645f93a0da0364978.jpg telah disimpan.
Label dan gambar untuk 452_jpg.rf.d83c5a5e061474284f0026eed386673f.jpg telah disimpan.
Label dan gambar untuk 452_jpg.rf.ed3b9bffcd6796c0ad1fb6a2c6c87dd4.jpg telah disimpan.
Label dan gambar untuk 452_jpg.rf.f199eade7e836fb729a97497b6e1064b.jpg telah disimpan.
Label dan gambar untuk 453_jpg.rf.4831ad129